## IP colab


In [ ]:
!curl ipecho.net/plain

##Puxando dados da API Mercado Livre

In [ ]:
import requests
import pandas as pd

# Termo de busca
termo_de_busca = input('Pesquise:')

# URL da API do Mercado Livre
url = f"(Link API| POR AQUI ){termo_de_busca}"

# Fazer a requisição GET
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Carregar os dados em formato JSON
    dados = response.json()

    # Extrair os dados relevantes
    produtos = []
    for item in dados.get('results', []):
        produto = {
            "Título": item.get('title'),
            "Preço (R$)": item.get('price'),
            "Link": item.get('permalink'),
            "Condição": item.get('condition'),
            "Categoria": item.get('category_id')
        }
        produtos.append(produto)

    # Criar o DataFrame
    df_mercado_livre = pd.DataFrame(produtos)
else:
    print(f"Erro na requisição: {response.status_code}")

## Tratamento simples, trocando os nomes.

In [ ]:
#Ordena por valor maior para o menor
df_mercado_livre = df_mercado_livre.sort_values(by= 'Preço (R$)', ascending=False)

# Trocando NEW por NOVO
df_mercado_livre['Condição'] = df_mercado_livre['Condição'].replace('new', 'Novo')
df_mercado_livre['Condição'] = df_mercado_livre['Condição'].replace('used', 'Usado')

#Troca dos nomes das colunas
df_mercado_livre.rename(columns={
                              'Título': 'produto',
                              'Link' : 'endereço_link',
                              'Condição' : 'condicao',
                              'Preço (R$)' : 'valor',
                              'Categoria' : 'categoria'
                              }, inplace=True)
#Remover o index
df_mercado_livre.reset_index().drop('index', axis = 1)



##Criação do arquivo CSV


In [ ]:
#Tratamento no Termo de busca e criação do arquivo CSV
termo_de_busca = termo_de_busca.strip().replace(" ", "_").lower()
df_mercado_livre.to_csv(f'mercado_livre_{termo_de_busca}_tratado.csv', index=False) #csv

## Envio do arquivo CSV para Bucket

In [ ]:
# Código com a chave
import os
from google.colab import auth
from google.cloud import storage

# Caminho da chave JSON da conta de serviço
chave_acesso = '=='
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = chave_acesso

project_id = 'projeto-joshua-bootcamp' #ID Projeto google cloud
!gcloud config set project {project_id}
client = storage.Client()
#NOME DA BUCKET
bucket_name = '=='
bucket = client.bucket(bucket_name)
#Destino do arquivo CSV, nome da pasta e nome do arquivo
destination_blob_name = (f'arquivos_tratados/mercado_livre_{termo_de_busca}_tratado.csv')
#nome do arquivo (DEVE SER IGUAL AO DE CIMA!!!)
source_file_name = (f'mercado_livre_{termo_de_busca}_tratado.csv')
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## Etapa SQL, envio e suas def de ddl e consulta

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
host = '==' # endereço do banco externo que eu quero acessar
user = '=='
password = '=='
database = '=='

###CÓDIGO

In [ ]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

def ddl(query):

    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(query)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{query}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

In [ ]:
ddl(f'''
CREATE TABLE mercado_livre_{termo_de_busca}(
id INT AUTO_INCREMENT PRIMARY KEY,
produto VARCHAR(255),
endereço_link VARCHAR(255),
condicao VARCHAR(50),
valor DECIMAL(10,2),
categoria VARCHAR(255)
);
''')

In [ ]:
csv_file = (f'/content/mercado_livre_{termo_de_busca}_tratado.csv')
df_SQL = pd.read_csv(csv_file)

tabela_nome = f'mercado_livre_{termo_de_busca}'
colunas = list(df_SQL.columns)
colunas_string = ', '.join(colunas)

#ordem : produto, valor, condicao, categoria, endereço_link
#dados_string = ', '.join( )

dados_list= []
for i in df_SQL.set_index('produto').itertuples():
  dados_list.append(tuple(i))


ddl(f'''
INSERT INTO {tabela_nome} ({colunas_string})
VALUES
  {', '.join([str(i) for i in dados_list])};
''')

In [ ]:
consulta(f'''
SELECT * FROM  mercado_livre_{termo_de_busca};
''',''' ''')

## Bigquery do Google cloud

In [ ]:
from google.colab import files
from google.cloud import bigquery
from google.colab import auth
import os


chave_acesso = '=='
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = chave_acesso

# Criar o cliente do BigQuery
client = bigquery.Client()


# Definir o ID do projeto e o ID da tabela
project_id = '`=='
table_id = (f'==.{termo_de_busca}')

df_bigquery = pd.read_csv(f'/content/mercado_livre_{termo_de_busca}_tratado.csv')

# Enviar o DataFrame para o BigQuery
df_bigquery.to_gbq(table_id, project_id=project_id, if_exists='replace')

##Envio do arquivo CSV para o MongoDB

In [ ]:
!pip install pymongo #Baixando pymongo

In [ ]:
from pymongo import MongoClient
# Conectar ao MongoDB Atlas
client = MongoClient('==')
db = client['==']

df_mongo = pd.read_csv(f'/content/mercado_livre_{termo_de_busca}_tratado.csv')
#Transforma em dicionario
df_mongo = df_mongo.to_dict(orient='records')
#Cria a colecao no MongoDb
colecao= db['==']
#enviar a coleção para o mongoDB
colecao.insert_many(df_mongo)